In [2]:
import fileio
import pydata

#import numpy as np
#import pandas as pd

In [27]:
from importlib import reload

In [164]:
reload(fileio)
#reload(shapes)

<module 'fileio' from '/home/nelson/Data/kba/fileio.py'>

In [161]:
myfile = fileio.InputData('dummy.csv')
myfile.iucnFile('dummycats.csv')

In [176]:
tilas = myfile.getTiles(0.2, offsetLat=0.1, offsetLon=0.1)

In [177]:
myfile.grid2shape('grid.shp')

In [141]:
myfile.originN

(-75.5, 5.5)

In [132]:
#ceil( ( (lon - self.originN[0]) / span[0]) * (totCols - 1))
#ceil( ( (self.originN[1] - lat) / span[1]) * (totRows - 1))
(-73 - -75.5) / 10.5 * (11 - 1) 

5.904761904761907

In [133]:
(-74 - -80.5) / 10.5 * (11 - 1) 

6.190476190476191

In [ ]:
myfile.sp

In [167]:
myfile = fileio.InputData('arecaceae.csv')
myfile.iucnFile('arecaceae_categories.csv')

In [ ]:
myar = shapes.KBA('/home/nelson/Dropbox/Humboldt/Postdoc/KBA_by_IUCN/Colombia_KBA', 'SitRecID')

In [ ]:
myar.spp_inclusion(myfile)

In [ ]:
myar.polys

In [ ]:
myar.new_spp_table('test_kba_log.csv')

---

In [ ]:
myfile.index_reg

In [5]:
for ti in tilas:
    print(myfile.tile2str(ti))

0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - 0 0 0 - - - 
- - - - - - 0 - 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - - 0 1 0 0 0 - - - 
- - - 0 0 1 1 1 0 0 - - 
- - - - 0 0 1 1 0 - - - 
- - - - 0 0 0 0 0 0 - - 
- - - - 0 0 0 0 - 0 - - 
- - - - 1 0 - 0 - - - - 
- - - - 0 0 - 0 - - - - 
- - 0 0 0 0 - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 

0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - 0 0 0 - - - 
- - - - - - 0 - 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - 0 0 0 1 0 1 1 - - 
- - - - 0 1 1 1 1 - - - 
- - - - 0 1 0 1 1 0 - - 
- - - - 0 1 1 1 - 0 - - 
- - - - 0 0 - 0 - - - - 
- - - - 0 1 - 0 - - - - 
- - 0 0 1 0 - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 

0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - 1 0 0 - - - 
- - - - - - 1 - 0 - - -

In [5]:
myfile.originN

(-81.3833333, 13.3575)

In [33]:
myfile.cellSize

1.0

In [41]:
myfile.index_reg.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [10]:
from shapely.geometry import Polygon, MultiPolygon, mapping
from shapely.ops import unary_union 

In [11]:
import fiona

In [21]:
schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int',
                  'score': 'float',
                  'NDMscore': 'float'},
    }

In [26]:
schema = {
    'geometry': 'MultiPolygon',
    'properties': {'id': 'int'},
    }

polys = []
for ic in range(mysols[0][0].getSize()):
    y, x = irkeys[ic]
    xBase = myfile.originN[0] + myfile.cellSize * (x + 1)
    yBase = myfile.originN[1] - myfile.cellSize * (y + 1)
    ocor = [(xBase, yBase + myfile.cellSize),
        (xBase - myfile.cellSize, yBase + myfile.cellSize),
        (xBase - myfile.cellSize, yBase),
        (xBase, yBase)]
    polys.append(Polygon(ocor))
    #solpoly = unary_union(polys)

multipol = MultiPolygon(polys)
    
with fiona.open('tila.shp', 'w', 'ESRI Shapefile', schema) as c:
    c.write({'geometry': mapping(multipol),
        'properties': {'id': 0}})

In [12]:
irkeys = list(myfile.index_reg.keys())

solpolcoll = []
multipol = None
for tsol in mysols[1]:
    polys = []
    solpoly = None
    for ic in range(tsol.getSize()):
        if tsol.getValue(ic) > 0:
            y, x = irkeys[ic]
            xBase = myfile.originN[0] + myfile.cellSize * x
            yBase = myfile.originN[1] - myfile.cellSize * y
            ocor = [(xBase, yBase + myfile.cellSize),
                (xBase - myfile.cellSize, yBase + myfile.cellSize),
                (xBase - myfile.cellSize, yBase),
                (xBase, yBase)]
            polys.append(Polygon(ocor))
            solpoly = unary_union(polys)
    solpolcoll.append(solpoly)
multipol = MultiPolygon(solpolcoll)

In [22]:
for ip, p in enumerate(solpolcoll):
     
    
    if ip == 0:
            with fiona.open('writing_test.shp', 'w', 'ESRI Shapefile', schema) as c:
                c.write({'geometry': mapping(p),
                    'properties': {'id': ip,
                                  'score':  mysols[1][ip].aggrScore,
                                  'NDMscore': mysols[1][ip].ndmScore}
                        })
    
    else:
        with fiona.open('writing_test.shp', 'a', 'ESRI Shapefile', schema) as c:
            c.write({'geometry': mapping(p),
                'properties': {'id': ip,
                                  'score':  mysols[1][ip].aggrScore,
                                  'NDMscore': mysols[1][ip].ndmScore}
                    })


In [17]:
mapping(p)

{'type': 'Polygon',
 'coordinates': (((-76.3833333, 9.3575),
   (-77.3833333, 9.3575),
   (-77.3833333, 10.3575),
   (-76.3833333, 10.3575),
   (-76.3833333, 11.3575),
   (-75.3833333, 11.3575),
   (-75.3833333, 10.3575),
   (-75.3833333, 9.3575),
   (-76.3833333, 9.3575)),)}

In [29]:
polys[0].bounds

(-76.3833333, 7.3575, -75.3833333, 8.3575)

In [30]:
polys[1].bounds

(-76.3833333, 6.3575, -75.3833333, 7.3575)

In [17]:
pol.bounds

(-76.3833333, 6.3575, -75.3833333, 7.3575)

In [24]:
polo.area

1.0

In [31]:
for spp in [tilas[x].getName() for x in mysols[0][0].spp2crit.keys()]:
    print(spp, len(myfile.points[spp]))

Wettinia hirsuta 16
Aiphanes lindeniana 31
Ceroxylon vogelianum 24
Chamaedorea ricardoi 11
Aiphanes leiostachys 2
Aiphanes parvifolia 11
Astrocaryum triandrum 6
Attalea cohune 2
Attalea nucifera 15
Bactris gasipaes 24
Ceroxylon quindiuense 30
Ceroxylon sasaimae 6
Cryosophila kalbreyeri 27
Geonoma bernalii 2
Geonoma chlamydostachys 12
Geonoma galeanoae 5
Hyospathe wendlandiana 5
Pholidostachys sanluisensis 4


In [26]:
print(myfile.tile2str( mysols[1][0]))

0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - 0 0 0 - - - 
- - - - - - 0 - 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - 0 0 0 1 0 0 0 - - 
- - - - 0 0 1 0 0 - - - 
- - - - 0 0 0 0 0 0 - - 
- - - - 0 0 0 0 - 0 - - 
- - - - 0 0 - 0 - - - - 
- - - - 0 0 - 0 - - - - 
- - 0 0 0 0 - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 



In [6]:
mysols = pydata.metasearchAlt(tilas, 0.2, 1000, 500, 2)

for ig, group in enumerate(mysols):
    print ('#' * 50)
    print ('Group {0}'.format(ig))
    for iso, so in enumerate(group):
        print("Solution {0} of {1}".format(iso, len(group)))
        print(so.score, "-", so.ndmScore, "-", so.aggrScore)
        #print("Islands: ", pydata.islNum(so))
        #if not pydata.isCont(so):
        #    print("NOT CONTINUOUS!!!")
        #print(so.toBitList())
        print(myfile.tile2str(so))
    

##################################################
Group 0
Solution 0 of 40
18 - 7.264217488467693 - 32.52843497693539
0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - 0 0 0 - - - 
- - - - - - 0 - 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - 0 0 0 0 0 0 0 - - 
- - - - 0 0 1 0 0 - - - 
- - - - 0 0 0 0 0 0 - - 
- - - - 0 0 0 0 - 0 - - 
- - - - 0 0 - 0 - - - - 
- - - - 0 0 - 0 - - - - 
- - 0 0 0 0 - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 
- - - - - - - - - - - - 
- - - - - - - - - - - 0 

Solution 1 of 40
8 - 3.9962733779102564 - 15.992546755820513
0 - - - - - - - - - - - 
- - - - - - - - - - - - 
- - - - - - 0 0 0 - - - 
- - - - - - 0 - 0 - - - 
- - - - 0 0 0 0 0 - - - 
- - - - 1 0 0 0 0 - - - 
- - - 0 0 0 0 0 0 0 - - 
- - - - 0 0 0 0 0 - - - 
- - - - 0 0 0 0 0 0 - - 
- - - - 0 0 0 0 - 0 - - 
- - - - 0 0 - 0 - - - - 
- - - - 0 0 - 0 - - - - 
- - 0 0 0 0 - - - - - - 
- - - - - - - - - - - - 
- - - - - - - - - -

In [ ]:
def overl(l0, l1):
    out = False
    for ic, c in enumerate(l0):
        if c == 1 and l1[ic] == 1:
            out = True
            break
    return out

In [ ]:
visited = [0 for x in mysols]
sets = {}
cc = 0

for isor, soRef in enumerate(mysols):
    if visited[isor] == 0:
        thisgroup = [isor]
        visited[isor] = 1
        for iso, so in enumerate(mysols):
            if iso != isor and visited[iso] == 0:
                addme = True
                for tg in thisgroup:
                    if overl(so.toBitList(), mysols[tg].toBitList()):
                        addme = False
                        break
                if addme:
                    visited[iso] = 1
                    thisgroup.append(iso)
        if len(thisgroup) > 1:
            sets[cc] = thisgroup
            cc += 1

In [ ]:
for iso in sets[0]:
    print(mysols[iso].score, "-", mysols[iso].ndmScore, "-", mysols[iso].aggrScore)
    print(myfile.tile2str(mysols[iso]))

In [ ]:
len(sets[0])

In [ ]:
[x for x in sets.keys() if sets[x] == 0]

In [ ]:
len(sets)

In [ ]:
mysols[0].originY

In [ ]:
mysols[0].originY = -90

In [ ]:
tarbits = [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
           [0, 0, 0, 1, 1, 1, 0, 0, 0, 0]]
counts = {0:0, 1:0, 2:0, 3:0}
#ndmabs = np.linspace(0, 50, 20).tolist()
#ndmouts = np.linspace(0, 1, 20).tolist()
ndmwei = np.linspace(10, 30, 20).tolist()
bffr = "index,weigth,ar0,ar1,ar2,ar3\n"
counter = 0

for w in ndmwei:            
    mysols = pyData.metasearchAlt(tilas, 0.2, 100, 20, w)
    tline = "{0},{1},".format(counter, w)
    for ita , ta in enumerate(tarbits):
        bit = "nan,"
        for iso, so in enumerate(mysols):
            if so.toBitList() == ta:
                counts[ita] += 1
                bit = "{0},".format(iso)
                break
        tline += bit
    tline = tline[:-1]
    bffr += tline + "\n"
    counter += 1
            
#with open('exp0.csv', 'w') as fhandle:
#with open('exp1.csv', 'w') as fhandle:
#    fhandle.write(bffr)

In [ ]:
print(bffr)

In [ ]:
exp0 = pd.read_csv('exp0.csv',index_col=0)

In [ ]:
exp0[['weigth', 'ar0', 'ar1', 'ar2', 'ar3']].groupby('weigth').mean()

In [ ]:
counts

/* keys are indexes in observations vector. Values are vectors integers that 
	associates to KBA criteria. Code options are:  
		0 = A1a  
		1 = A1b  
		2 = A1c  
		3 = A1d  
		4 = A1e  
		5 = B1  
		6 = B2*/ 

In [ ]:
myfile.rows, myfile.cols

In [ ]:
for r in range(myfile.rows):
    for c in range(myfile.cols):
        print("{0:.1f}|".format(myfile.presence_grid[r][c]), end='')
    print('')